# ✨ [Day 8](https://adventofcode.com/2021/day/8)

In [1]:
import numpy as np
from collections import defaultdict


def parse_inputs(x):
    lines = x.splitlines()
    inputs, outputs = [], []
    for l in lines:
        if not l: continue
        i, o = l.split(' | ')
        inputs.append(i.split())
        outputs.append(o.split())
    return inputs, outputs


def match_pattern(x):
    """Mapp a code entry to possible numbers"""
    if len(x) == 7:
        return [8]
    elif len(x) == 6:
        return [0, 6, 9]
    elif len(x) == 5:
        return [2, 3, 5]
    elif len(x) == 4:
        return [4]
    elif len(x) == 3:
        return [7]
    elif len(x) == 2:
        return [1]
    else:
        return []
    
def find_unique_patterns(lines):
    """Part 1"""
    return sum(len(match_pattern(x)) == 1 for l in lines for x in l)

def determine_mapping(seq, outputs):
    """Given a sequence of codes, determine the segment mapping"""
    # Add constraints
    assignment = defaultdict(lambda: [])
    for code in seq:
        for possibility in match_pattern(code):
            assignment[possibility].append(code)
            
    # 1 4 7 and 8 are solved
    for key in [1, 4, 7, 8]:
        assignment[key] = assignment[key][0]
    
    # disambiguate [2, 3, 5] case
    # only "3" contains the segments of "1"
    assignment[3] = [code for code in assignment[3] if all(c in code for c in assignment[1])]
    assert len(assignment[3]) == 1
    assignment[3] = assignment[3][0]
    # "5" and "3" contains 3 segments of "4", while "2" only contains 2
    assignment[2] = min(assignment[2], key=lambda code: sum(c in code for c in assignment[4]))
    # leftover: 5
    assignment[5] = [x for x in assignment[5] if x not in [assignment[2], assignment[3]]]
    assert len(assignment[5]) == 1
    assignment[5] = assignment[5][0]
            
    # disambiguate [0, 6, 9] case
    # "9" is the only one that contains all segmentsof "4"
    assignment[9] = [code for code in assignment[9] if all(c in code for c in assignment[4])]
    assert len(assignment[9]) == 1
    assignment[9] = assignment[9][0]
    # "6" contains all segments of "5" while "0" does not
    assignment[6] = [code for code in assignment[6] if (
        all(c in code for c in assignment[5]) and code != assignment[9])]
    assert len(assignment[6]) == 1
    assignment[6] = assignment[6][0]
    # leftover : 0
    assignment[0] = [x for x in assignment[0] if x not in [assignment[6], assignment[9]]]
    assert len(assignment[0]) == 1
    assignment[0] = assignment[0][0]
    
    # Get final output number
    rev_index = {tuple(sorted(v)): k for k, v in assignment.items()}
    x = 0
    for o in outputs:
        num = rev_index[tuple(sorted(o))]
        x = 10 * x + num
    return x


def get_output_sum(inputs, outputs):
    return sum(determine_mapping(x, y) for x, y in zip(inputs, outputs))

In [2]:
with open('inputs/day08.txt', 'r') as f:
    inputs = f.read()
    

inputs, outputs = parse_inputs(inputs)
print(f"There are \033[92;1m{find_unique_patterns(outputs)}\033[0m unique patterns (1, 4, 7, 8)"
      " in the diplayed numbers")
print(f"The sum of decoded outputs is \033[92;1m{get_output_sum(inputs, outputs)}\033[0m")

There are 488 unique patterns (1, 4, 7, 8) in the diplayed numbers
The sum of decoded outputs is 1040429
